# web-monitoring backend demo

1. Ingest a cache of captured HTML files, representing a **Page** as a series of **Versions** through time.
2. Two Versions of the same Page are a **Change**.
3. To examine a given Change, start by sending requests to PageFreezer. Store its respones (**Diffs**).
4. Assign a Priority to each Change.
4. Access prioritized Changes and store user-submitted **Annotations** (potentially multiple Annotations per Change.)

In [1]:
from datetime import datetime, timedelta
import functools
import hashlib
import os

import sqlalchemy
from web_monitoring.db import (Pages, Versions, Diffs, Annotations, create,
                               compare, NoAncestor, diff_version, logger)

engine = sqlalchemy.create_engine(os.environ['WEB_MONITORING_SQL_DB_URI'])

In [2]:
create(engine)  # one time only: create tables

# Reflect SQL tables in Python.
versions = Versions(engine)
pages = Pages(engine)
diffs = Diffs(engine)
annotations = Annotations(engine)

## Ingesting new HTML

Either manually or via some webhook, the backend is alerted that new captured HTML is avaialbe at some path.

In this example, we load the example files in the web-monitoring repo.

In [3]:
def load_examples():
    EXAMPLES = [
        'falsepos-footer',
        'falsepos-num-views',
        'falsepos-small-changes',
        'truepos-dataset-removal',
        'truepos-image-removal',
        'truepos-major-changes',
    ]
    archives_dir = os.path.join('archives')
    time1 = datetime.now()
    time0 = time1 - timedelta(days=1)
    for example in EXAMPLES:
        simulated_url = 'https://PLACEHOLDER.com/{}.html'.format(example)
        page_uuid = pages.insert(simulated_url, 'some page title', 'some agency', 'some site')
        for suffix, _time in (('-a.html', time0), ('-b.html', time1)):
            filename = example + suffix
            path = os.path.abspath(os.path.join(archives_dir, filename))
            with open(path) as f:
                version_hash = hashlib.sha256(str(f.read()).encode()).hexdigest()
            versions.insert(page_uuid, _time, path, version_hash, 'test', {})
            
load_examples()

No we have a pile of unprocessed Snapshots. Some might be the first time we have seen a Page, while others might be just another Snapshot of a Page we have seen before.

In [4]:
versions.unprocessed

deque(['129918b1-43a7-46dd-a7c3-bf1cff06d043',
       '4403cbe4-4b1b-4fa9-b481-37b7b8bc8127',
       'd9ef4261-7340-4969-a823-3dd89d6c2565',
       '032252ac-fbb1-4994-892d-4d88dd28fd19',
       'ca981bb8-c0d7-4538-b801-fa9c5ed07e2c',
       'bad7db40-3996-42f5-81b4-d4b816185c90',
       '6e70f0e7-a931-4f8e-a73d-ca1d3a212dec',
       'cb0aff43-b50c-42ed-8d25-71b99d8ba7f5',
       '023ae577-87a1-46e2-af64-f233183f9a76',
       '5ab820a7-73ec-4aee-be44-ecf4cc61f514',
       '2996f0e7-3631-4c10-b6b5-784d2d98eaa4',
       '62ffe2ed-9ffd-4b3c-a307-c5cfd8308bdd'])

The Python API provides uuid-based lookup and returns the data as a `namedtuple` (low memory footprint, convenient attribute access).

In [5]:
v = versions[versions.unprocessed[0]]
v

Version(uuid='129918b1-43a7-46dd-a7c3-bf1cff06d043', page_uuid='4e591197-9475-4f2b-a0ea-3331809d8ce5', capture_time=datetime.datetime(2017, 3, 21, 17, 28, 4, 459319), uri='/Users/dallan/Documents/Repos/web-monitoring-processing/archives/falsepos-footer-a.html', version_hash='41af79e31884c6745834961f435cf233de702065b6bba032a82ec68fc5fd03b7', source_time='test', source_metadata={})

In [6]:
pages[v.page_uuid]

Page(uuid='4e591197-9475-4f2b-a0ea-3331809d8ce5', url='https://PLACEHOLDER.com/falsepos-footer.html', title='some page title', agency='some agency', site='some site')

## Computing Diffs between Snapshots

Iterate through the unprocessed Snapshots and requests diffs from PageFreezer. Stash the JSON response (which is large) in a file on disk. Store the filepath, the two Snapshots' UUIDs, and other small summary info in the database.

In [7]:
# Set up standard Python logging.
import logging
logging.basicConfig(level='DEBUG')
# This logger will show progress with PageFreezer requests.
logger.setLevel('DEBUG')

def diff_new_versions():
    f = functools.partial(diff_version, versions=versions, diffs=diffs,
                          source_type='test', source_metadata={})
    while True:
        # Get the uuid of a Version to be processed.
        try:
            version_uuid  = versions.unprocessed.popleft()
        except IndexError:
            # nothing left to process
            return
        try:
            f(version_uuid)
        except NoAncestor:
            # This is the oldest Version for this Page -- nothing to compare.
            continue

diff_new_versions()

Logger output:
```
DEBUG:web_monitoring.db:Sending PageFreezer request...
DEBUG:web_monitoring.db:Response received in 6.507 seconds with status ok.
DEBUG:web_monitoring.db:Sending PageFreezer request...
DEBUG:web_monitoring.db:Response received in 9.260 seconds with status ok.
DEBUG:web_monitoring.db:Sending PageFreezer request...
DEBUG:web_monitoring.db:Response received in 2.576 seconds with status ok.
DEBUG:web_monitoring.db:Sending PageFreezer request...
DEBUG:web_monitoring.db:Response received in 13.063 seconds with status ok.
DEBUG:web_monitoring.db:Sending PageFreezer request...
DEBUG:web_monitoring.db:Response received in 2.529 seconds with status ok.
DEBUG:web_monitoring.db:Sending PageFreezer request...
DEBUG:web_monitoring.db:Response received in 2.448 seconds with status ok.
```

Now we have Diffs that need to be prioritized.

Accessing the diff from the Python API access that stashed JSON file and transparently fills it into the result. Since it's quite verbose, we'll just look at the *fields* here, not the values.

In [8]:
diffs[diffs.unprocessed[0]]._fields

('uuid',
 'version_from',
 'version_to',
 'diffhash',
 'uri',
 'source_type',
 'source_metadata',
 'content')

## Prioritizing Diffs

Iterate through the unprocessed Diffs and assign a priority. This is where the clever text processing code would come in.

In [9]:
def assign_priorities(diff_uuids):
    priorities = {}
    for diff_uuid in diff_uuids:
        d = diffs[diff_uuid]
        priority = 0  # replace this with:  priority = clever_ML_routine(d)
        priorities[diff_uuid] = priority
    return priorities

assign_priorities(diffs.unprocessed)

{'0295a42f-01ae-4273-8783-cf73fc912a9c': 0,
 '1046caa6-07a2-46c6-9737-d3da23194909': 0,
 '7b33801d-553f-476e-bdf2-690f07511c80': 0,
 '97d95f38-dd18-4eca-8093-6d31df792507': 0,
 'f6c3c688-a8fd-40a7-b369-f74169bb8c7d': 0,
 'f9271b3b-26c9-4045-aa19-38b88711d6f5': 0}